pip install liftover

https://pypi.org/project/liftover/m

In [1]:
import pandas as pd
from liftover import get_lifter

def read_regenie(file):
    try:
        sumstats = pd.read_csv(file, compression='gzip', header=0, sep='\t', quotechar='"')
    except:
        sumstats = pd.read_csv(file, header=0, sep='\t', quotechar='"')
    sumstats.SNP = 'chr'+sumstats.CHR.astype(str) + ':' + sumstats.POS.astype(str) + ':' + sumstats.REF.astype(str) + ':' + sumstats.ALT.astype(str)
    sumstats.CHR = sumstats.CHR.astype(int)
    sumstats.POS = sumstats.POS.astype(int)
    return sumstats

def sumstat_liftover(ss,fr='hg19' ,to ='hg38'):
    chainmap = get_lifter(fr, to)
    lchr,lpos,lsnp = [],[],[]
    for c,p,a0,a1 in zip(ss.CHR,ss.POS,ss.REF,ss.ALT):
        try:
            new_c,new_p,_ = chainmap[int(c)][p][0]
            snp = ':'.join([new_c,str(new_p),a0,a1])
            new_c = int(new_c[3:])
        except:
            new_c, new_p = 0, 0
            snp = ':'.join(['chr'+str(c),str(p),a0,a1])
        lchr.append(new_c)
        lpos.append(new_p)
        lsnp.append(snp)
    ss.CHR = lchr
    ss.POS = lpos
    ss.SNP = lsnp
    return ss

def main(input_path,output_path,remove_missing):
    sums = read_regenie(input_path)
    sums1 = sumstat_liftover(sums)
    if remove_missing:
        sums1[sums1.CHR == 0].to_csv(output_path, sep = "\t", header = True, index = False)
    else:
        sums1.to_csv(output_path, sep = "\t", header = True, index = False)

In [2]:
input_path = '/home/dmc2245/UKBiobank/results/REGENIE_results/results_imputed_data/2021_10_07_combined_500K/100521_UKBB_Combined_f2247_f2257_expandedwhite_93258cases_237318ctrl_500k_PC1_PC2_f2247_f2257.regenie.snp_stats.gz'
output_path = '/home/dmc2245/test/sumstats.test'
remove_missing = True

In [3]:
main(input_path,output_path,remove_missing)

ConnectionError: HTTPSConnectionPool(host='hgdownload.cse.ucsc.edu', port=443): Max retries exceeded with url: /goldenPath/hg19/liftOver/hg19ToHg38.over.chain.gz (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x2ac35aed38b0>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [4]:
from liftover import get_lifter

converter = get_lifter('hg19', 'hg38')
chrom = '1'
pos = 103786442
converter[chrom][pos]


ConnectionError: HTTPSConnectionPool(host='hgdownload.cse.ucsc.edu', port=443): Max retries exceeded with url: /goldenPath/hg19/liftOver/hg19ToHg38.over.chain.gz (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x2ac3e47c56a0>: Failed to establish a new connection: [Errno 110] Connection timed out'))